## What is a Grid

A grid is a form of spatial data that represents information (such as a gravity intensity, a magnetic reading, or a colour) at points organized as a 2-dimensional array on a right-handed cartesian plane:

<img src="https://github.com/GeosoftInc/gxpy/raw/9.3/examples/tutorial/Grids%20and%20Images/image2017-6-14_13-9-19.png" alt="Drawing" style="width: 500px;"/>

The plane on which the grid is located can be oriented in three dimensions relative to a Coordinate System on the Earth.  The most common grids are located on a horizontal surface relative to the Coordinate System.  For example, common surfaces might be sea-level, or the ground surface, or a constant elevation above or below the ground surface, or a constant elevation. Vertical cross-sections through the Earth are oriented to be orthogonal to the surface of the Earth.

A common term used with grids is the concept of a 'grid cell'.  In Geosoft's usage, grids are an array of points at a point location, and a 'grid cell' is the rectangular area that extends half-way to the neighboring grid points.

Spatial reference angles throughout the _geosoft.gxpy_ module will consistently use an angle in degrees azimuth, which is a clockwise-positive angle relative to a coordinate system frame (North, or positive Y). The _geosoft.gxapi.GXIMG_ class specifies the rotation angle in degrees counterclockwise-positive, and other references within the geosoft.gxapi may differ.

Grids are stored as files on the file system, and there are many common grid file formats in existence. When working with grid files in GX Developer you define the grid file format with the use of a decorator string appended to the grid file name. Geosoft supports the 11 formats (and their many derivatives) described in the Grid File Name Decorations section of the GX Developer documentation.  For example:

| grid file string | grid file type |
|:----------------:|:-------------- |
| 'c:/project/mag.grd(GRD)'	| Geosoft format grid. |
| 'c:/project/mag.tif(TIF)'	| GeoTIF |
| 'c:/project/image.jpg(IMG;T=5)' | jpeg image file
| 'c:/project/mag.grd(GRD;TYPE=COLOR)' | Geosoft colour grid |

__See also:__ [Tutorial Page](https://geosoftgxdev.atlassian.net/wiki/spaces/GXD93/pages/103415893/Grids+and+Images)

## Convert a grid from one format to another
__Problem:__ You have a grid in a Geosoft-supported format, and you need the grid in some other format to use in a different application.

__Grid:__ elevation_surfer.grd, which is a Surfer v7 format grid file.

### Get Data

In [ ]:
import geosoft.gxpy.gx as gx
import geosoft.gxpy.utility as gxu
gxc = gx.GXpy()
url = 'https://github.com/GeosoftInc/gxpy/raw/9.3/examples/tutorial/Grids%20and%20Images/'
gxu.url_retrieve(url + 'elevation_surfer.GRD')

__Approach:__

1. Open the surfer grid with decoration '(SRF;VER=V7)', as documented in Grid File Name Decorations.
2. Use the _gxgrid.Grid.copy_ class method to create an ER Mapper grid, which will have decoration '(ERM)'.

In [ ]:
import geosoft.gxpy.gx as gx
import geosoft.gxpy.grid as gxgrid

# create context
gxc = gx.GXpy()

# open grid
grid_surfer = gxgrid.Grid.open('elevation_surfer.grd(SRF;VER=V7)')

# copy the grid to an ER Mapper format grid file
grid_erm = gxgrid.Grid.copy(grid_surfer, 'elevation.ers(ERM)', overwrite=True)

## Grid Coordinate System

In Geosoft all spatial data should have a defined coordinate system which allows data to be located on the Earth.  This also takes advantage of Geosoft's ability to reproject data as required.  However, in this example the Surfer grid does not store the coordinate system information, but we know that the grid uses projection 'UTM zone 54S' on datum 'GDA94'.   Let's modify this script to set the coordinate system, which will be saved as part of the ER Mapper grid, which does have the ability to store  the coordinate system description.

In Geosoft, well-known coordinate systems like this can be described using the form 'GDA94 / UTM zone 54S', which conforms to the SEG Grid Exchange Format standard for describing coordinate systems.  You only need to set the coordinate_system property of the grid_surfer instance.

In [ ]:
import geosoft.gxpy.gx as gx
import geosoft.gxpy.grid as gxgrid
 
# create context
gxc = gx.GXpy()
 
# open grid
grid_surfer = gxgrid.Grid.open('elevation_surfer.grd(SRF;VER=V7)', mode=gxgrid.FILE_READWRITE)
 
# define the coordinate system
grid_surfer.coordinate_system = 'GDA94 / UTM zone 54S'
 
# copy the grid to an ER Mapper format grid file
grid_erm = gxgrid.Grid.copy(grid_surfer, 'elevation.ers(ERM)', overwrite=True)
 
print(str(grid_erm.coordinate_system))

Coordinate systems also contain the full coordinate system parameter information, from which you can construct coordinate systems in other applications.

In [ ]:
grid_erm.coordinate_system.gxf

In [ ]:
grid_erm.coordinate_system.esri_wkt

In [ ]:
grid_erm.coordinate_system.json

## Basic Grid Statistics

In this exercise we will work with the data stored in a grid.  One common need is to determine some basic statistical information about the grid data, such as the minimum, maximum, mean and standard deviation.  This exercise will work with the grid data a number of ways that demonstrate some useful patterns.

### Statistics using numpy

The smallest code and most efficient approach is to read the grid into a numpy array and then use the optimized numpy methods to determine statistics.  This has the benefit of speed and simplicity at the expense memory, which may be a concern for very large grids, though on modern 64-bit computers with most grids this would be the approach of choice. 

In [ ]:
# these lines release the resources for the previously open grids
grid_surfer = None
grid_erm = None

# open the grid
with gxgrid.Grid.open('elevation_surfer.grd(SRF;VER=V7)') as grid:
 
    # get the data in a numpy array
    data_values = grid.xyzv()[:, :, 3]
 
# print statistical properties
print('minimum: ', np.nanmin(data_values))
print('maximum: ', np.nanmax(data_values))
print('mean: ', np.nanmean(data_values))
print('standard deviation:', np.nanstd(data_values))

### Statistics using Geosoft VVs

Many Geosoft methods will work with a _geosoft.gxpy.vv.GXvv_, which wraps the _geosoft.gxapi.GXVV_ class that deals with very long single-value vectors.  The Geosoft _GXVV_ methods works with Geosoft data types and, like numpy, is optimized to take advantage of multi-core processors to improve performance.  The pattern in this exercise reads a grid one grid row at a time, returning a _GXvv_  instance and accumulate statistics in an instance of the [_geosoft.gxapi.GXST_](http://localhost:63342/gxpy/docs/_build/html/GXST.html) class.

In [ ]:
import geosoft.gxapi as gxapi

# create a gxapi.GXST instance to accumulate statistics
stats = gxapi.GXST.create()
 
# open the grid
with gxgrid.Grid.open('elevation_surfer.grd(SRF;VER=V7)') as grid:
 
    # add data from each row to the stats instance
    for row in range(grid.ny):
        stats.data_vv(grid.read_row(row).gxvv)
 
# print statistical properties
print('minimum: ', stats.get_info(gxapi.ST_MIN))
print('maximum: ', stats.get_info(gxapi.ST_MAX))
print('mean: ', stats.get_info(gxapi.ST_MEAN))
print('standard deviation:', stats.get_info(gxapi.ST_STDDEV))

## Grid Iterator

A grid instance also behaves as an iterator that works through the grid data points by row, then by column, each iteration returning the (x, y, z, grid_value).  In this example we will iterate through all points in the grid and accumulate the statistics a point at a time.  This is the least-efficient way to work through a grid, but the pattern can be useful to deal with a very simple need.  For example, any Geosoft supported grid can be easily converted to an ASCII file that has lists the (x, y, z, grid_value) for all points in a grid.

In [ ]:
# create a gxapi.GXST instance to accumulate statistics
stats = gxapi.GXST.create()
 
# add each data to stats point-by-point (slow, better to use numpy or vector approach)
number_of_dummies = 0
with gxgrid.Grid.open('elevation_surfer.grd(SRF;VER=V7)') as grid:
    for x, y, z, v in grid:
        if v is None:
            number_of_dummies += 1
        else:
            stats.data(v)
    total_points = grid.nx * grid.ny
 
# print statistical properties
print('minimum: ', stats.get_info(gxapi.ST_MIN))
print('maximum: ', stats.get_info(gxapi.ST_MAX))
print('mean: ', stats.get_info(gxapi.ST_MEAN))
print('standard deviation:', stats.get_info(gxapi.ST_STDDEV))
print('number of dummies: ', number_of_dummies)
print('number of valid data points: ', total_points - number_of_dummies)